# Importing necessary libraries

In [1]:
#import necessary dependecies
import seaborn as sns
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, AdaBoostClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.cluster import KMeans
from sklearn.preprocessing import LabelEncoder
from sklearn.pipeline import Pipeline
import os
import warnings
warnings.filterwarnings('ignore')
import numpy as np  
import pandas as pd, os, gc
import matplotlib.pyplot as plt
from sklearn.metrics import roc_curve, auc, log_loss
from sklearn.feature_selection import mutual_info_classif
from sklearn.preprocessing import StandardScaler, RobustScaler
%matplotlib inline
# from typing import List

C:\Users\SHOLAY\Anaconda3\lib\site-packages\statsmodels\tools\_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


# Environment setup

In [2]:
pd.set_option('display.max_columns', 50)
pd.set_option('display.max_rows', 50)

# Dataset

In [3]:
trains = 'Train.csv'
tests = 'Test.csv'
submissions = 'SampleSubmission.csv'

# Function to load data

In [4]:
def read_data(data, parse_date=None):
    if parse_date is not None:
        return pd.read_csv(data, parse_dates=[parse_date])
    else:
        return pd.read_csv(data)

# Loading the datasets

In [5]:
train = read_data(trains, parse_date='join_date')
test = read_data(tests, parse_date='join_date')
submission= read_data(submissions)
del trains, tests, submissions

# Data overview

In [6]:
train.head()

,ID,join_date,sex,marital_status,birth_year,branch_code,occupation_code,occupation_category_code,P5DA,RIBP,8NN1,7POT,66FJ,GYSR,SOP4,RVSZ,PYUQ,LJR9,N2MW,AHXO,BSTQ,FM3X,K6QO,QBOL,JWFN,JZ9D,J9JW,GHYX,ECY3
0,4WKQSBB,2019-01-02,F,M,1987,1X1H,2A7I,T4MS,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0
1,CP5S02H,2019-01-06,F,M,1981,UAOD,2A7I,T4MS,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0
2,2YKDILJ,2013-01-06,M,U,1991,748L,QZYX,90QI,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,1
3,2S9E81J,2019-01-08,M,M,1990,1X1H,BP09,56SI,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0
4,BHDYVFT,2019-01-08,M,M,1990,748L,NO3L,T4MS,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0


In [7]:
test.head()

,ID,join_date,sex,marital_status,birth_year,branch_code,occupation_code,occupation_category_code,P5DA,RIBP,8NN1,7POT,66FJ,GYSR,SOP4,RVSZ,PYUQ,LJR9,N2MW,AHXO,BSTQ,FM3X,K6QO,QBOL,JWFN,JZ9D,J9JW,GHYX,ECY3
0,F86J5PC,2018-01-12,M,M,1984,94KC,DZRV,90QI,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0
1,H6141K3,2019-01-10,M,M,1996,1X1H,J9SY,90QI,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0
2,RBAYUXZ,2020-01-01,F,W,1968,UAOD,2A7I,T4MS,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0
3,KCBILBQ,2019-01-02,M,M,1989,94KC,2A7I,T4MS,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0
4,LSEC1ZJ,2020-01-02,F,M,1982,UAOD,0KID,T4MS,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0


In [8]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 29132 entries, 0 to 29131
Data columns (total 29 columns):
 #   Column                    Non-Null Count  Dtype         
---  ------                    --------------  -----         
 0   ID                        29132 non-null  object        
 1   join_date                 29130 non-null  datetime64[ns]
 2   sex                       29132 non-null  object        
 3   marital_status            29132 non-null  object        
 4   birth_year                29132 non-null  int64         
 5   branch_code               29132 non-null  object        
 6   occupation_code           29132 non-null  object        
 7   occupation_category_code  29132 non-null  object        
 8   P5DA                      29132 non-null  int64         
 9   RIBP                      29132 non-null  int64         
 10  8NN1                      29132 non-null  int64         
 11  7POT                      29132 non-null  int64         
 12  66FJ              

# Checking the data

In [9]:
print(f'Total columns for training set: {len(train.columns)}');print(f'Total columns for testing set: {len(train.columns)}')

Total columns for training set: 29
Total columns for testing set: 29


In [10]:
train.columns == test.columns

array([ True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True])

In [11]:
#print all dataset shape
print(f'Train shape: {train.shape}\nTest shape:  {test.shape}\nSubmission shape: {submission.shape}')

Train shape: (29132, 29)
Test shape:  (10000, 29)
Submission shape: (210000, 2)


In [12]:
#Store Products ID in Multidimensional array
col= ['P5DA', 'RIBP', '8NN1',
       '7POT', '66FJ', 'GYSR', 'SOP4', 'RVSZ', 'PYUQ', 'LJR9', 'N2MW', 'AHXO',
       'BSTQ', 'FM3X', 'K6QO', 'QBOL', 'JWFN', 'JZ9D', 'J9JW', 'GHYX', 'ECY3']
products = train[col]
products_t = test[col]
del col

In [13]:
#Axis Transformation
train = train.melt(id_vars=train.columns[:8], value_vars=products, var_name = "PCODE", value_name="Label" )
test = test.melt(id_vars=test.columns[:8], value_vars=products, var_name = "PCODE", value_name="Label" )

In [14]:
#A merger
train['combiner']='x'

In [15]:
#Set
data=pd.concat([train,test],sort=False).reset_index(drop=True)

In [16]:
#preview random 3 sample
data.sample(3)

,ID,join_date,sex,marital_status,birth_year,branch_code,occupation_code,occupation_category_code,PCODE,Label,combiner
672244,WIW4H0O,2018-01-03,F,M,1972,748L,SST3,56SI,SOP4,0,NaN
678978,YDJECMF,2018-01-03,F,M,1985,748L,RUFT,T4MS,SOP4,0,NaN
614601,J3PM30H,2018-01-04,M,M,1975,30H5,2A7I,T4MS,P5DA,0,NaN


In [17]:
#Confirm Distribution (TEST AND SAMPLE SUBMISSION FILE)
print('==' * 18);print('TRAIN LABEL DISTRIBUTION');print('==' * 18);print(train['Label'].value_counts())
print('==' * 18);print('TEST LABEL DISTRIBUTION');print('==' * 18);print(test['Label'].value_counts())
print('==' * 18);print('SAMPLE SUBMISSION LABEL DISTRIBUTION');print('==' * 18);print(submission['Label'].value_counts())

TRAIN LABEL DISTRIBUTION
0    545419
1     66353
Name: Label, dtype: int64
TEST LABEL DISTRIBUTION
0    197147
1     12853
Name: Label, dtype: int64
SAMPLE SUBMISSION LABEL DISTRIBUTION
0    197147
1     12853
Name: Label, dtype: int64


In [18]:
train=data[data.combiner.notnull()].reset_index(drop=True)
test=data[data.combiner.isna()].reset_index(drop=True)
train.drop('combiner', inplace=True, axis=1)
test.drop(['Label','combiner'], inplace=True, axis=1)
print(train.shape, test.shape, submission.shape)

(611772, 10) (210000, 9) (210000, 2)


In [19]:
#Concatenate new column towards submission requirement
train['ID X PCODE'] = train['ID'] + ' X ' + train['PCODE']
test['ID X PCODE'] = test['ID'] + ' X ' + test['PCODE']

In [20]:
#Re-arrange column (not necessary)
col = ['ID', 'join_date', 'sex', 'marital_status', 'birth_year', 'branch_code',\
       'occupation_code', 'occupation_category_code', 'PCODE','ID X PCODE']
test = test[col]
col.append('Label')
train = train[col]
del col

In [21]:
train.head()

,ID,join_date,sex,marital_status,birth_year,branch_code,occupation_code,occupation_category_code,PCODE,ID X PCODE,Label
0,4WKQSBB,2019-01-02,F,M,1987,1X1H,2A7I,T4MS,P5DA,4WKQSBB X P5DA,0
1,CP5S02H,2019-01-06,F,M,1981,UAOD,2A7I,T4MS,P5DA,CP5S02H X P5DA,0
2,2YKDILJ,2013-01-06,M,U,1991,748L,QZYX,90QI,P5DA,2YKDILJ X P5DA,0
3,2S9E81J,2019-01-08,M,M,1990,1X1H,BP09,56SI,P5DA,2S9E81J X P5DA,0
4,BHDYVFT,2019-01-08,M,M,1990,748L,NO3L,T4MS,P5DA,BHDYVFT X P5DA,0


In [22]:
test.head()

,ID,join_date,sex,marital_status,birth_year,branch_code,occupation_code,occupation_category_code,PCODE,ID X PCODE
0,F86J5PC,2018-01-12,M,M,1984,94KC,DZRV,90QI,P5DA,F86J5PC X P5DA
1,H6141K3,2019-01-10,M,M,1996,1X1H,J9SY,90QI,P5DA,H6141K3 X P5DA
2,RBAYUXZ,2020-01-01,F,W,1968,UAOD,2A7I,T4MS,P5DA,RBAYUXZ X P5DA
3,KCBILBQ,2019-01-02,M,M,1989,94KC,2A7I,T4MS,P5DA,KCBILBQ X P5DA
4,LSEC1ZJ,2020-01-02,F,M,1982,UAOD,0KID,T4MS,P5DA,LSEC1ZJ X P5DA


In [23]:
#Lets Check for data co-existence in Train/Test
for col in ['PCODE','occupation_code', 'branch_code','occupation_category_code']:
    train_col = train[col].unique();test_col = test[col].unique()
    print ('Not in Test ' + col + ' ',[i  for i in train_col if i not in test_col])
    print ('Not in Train ' + col + ' ',[i  for i in test_col  if i not in train_col]);print('==' * 30)

Not in Test PCODE  []
Not in Train PCODE  []
Not in Test occupation_code  ['IE90', 'Q0LY', 'JSAX', '6XXU', '8HRZ', 'OQMY', 'INEJ', 'VZN9', 'UC7E', 'PSUY', 'WSRG', 'JQH3', 'LGTN', '738L', 'QQUP', '2XZ1', 'CAAV', 'LLLH', 'W1X2', 'DHSN', 'IX8T', '2US6', 'ZWPL', 'MEFQ', '9B5B', 'JUIP', 'BFD1', 'A4ZC', 'IMHI', 'E5PF', 'GZA8', '3YQ1', 'PJR4', 'NDL9', 'PPNK', '2686', '5LNN', '374O', 'URYD', 'M0WG', 'KBWO', 'ONY7', 'VYSA', 'KUPK', 'R7GL', 'HSVE', 'BER4', '6SKY', 'RH2K', 'ZHC2', 'W3ZV', 'FLXH', 'UYDZ', 'YJXM', '59QM']
Not in Train occupation_code  ['0ZND', '8CHJ', '9F96', 'HSI5', '93OJ', 'BIA0', 'E2MJ', 'JBJP', '0FOI']
Not in Test branch_code  []
Not in Train branch_code  []
Not in Test occupation_category_code  []
Not in Train occupation_category_code  []


In [24]:
train.tail()

,ID,join_date,sex,marital_status,birth_year,branch_code,occupation_code,occupation_category_code,PCODE,ID X PCODE,Label
611767,8SBA93U,2019-01-09,F,M,1974,748L,2A7I,T4MS,ECY3,8SBA93U X ECY3,0
611768,Y73ETTO,2018-01-11,M,M,1987,XX25,Q6J6,90QI,ECY3,Y73ETTO X ECY3,0
611769,MVXG2LC,2019-01-02,M,M,1985,BOAS,6PE7,T4MS,ECY3,MVXG2LC X ECY3,0
611770,BUQ8DQN,2019-01-07,M,M,1989,UAOD,2A7I,T4MS,ECY3,BUQ8DQN X ECY3,0
611771,DEJBWYU,2020-01-03,M,M,1975,94KC,31JW,90QI,ECY3,DEJBWYU X ECY3,0


In [25]:
test.tail()

,ID,join_date,sex,marital_status,birth_year,branch_code,occupation_code,occupation_category_code,PCODE,ID X PCODE
209995,0GMU5UH,2019-01-04,F,M,1986,748L,2A7I,T4MS,ECY3,0GMU5UH X ECY3
209996,I9W11CD,2019-01-08,M,M,1980,UAOD,0KID,T4MS,ECY3,I9W11CD X ECY3
209997,42WTEGT,2018-01-04,M,M,1993,748L,SST3,56SI,ECY3,42WTEGT X ECY3
209998,8EKC4O9,2015-01-05,F,U,1981,49BM,2A7I,T4MS,ECY3,8EKC4O9 X ECY3
209999,WHII93K,2018-01-11,M,U,1977,94KC,2A7I,T4MS,ECY3,WHII93K X ECY3


In [26]:
# LABEL ENCODE
def encode_LE(train,test,cols,verbose=True):
    for col in cols:
        df_comb = pd.concat([train[col],test[col]],axis=0)
        df_comb,_ = df_comb.factorize(sort=True)
        nm = col
        if df_comb.max()>32000: 
            train[nm] = df_comb[:len(train)].astype('int32')
            test[nm] = df_comb[len(train):].astype('int32')
        else:
            train[nm] = df_comb[:len(train)].astype('int16')
            test[nm] = df_comb[len(train):].astype('int16')
        del df_comb; x=gc.collect()
        if verbose: print(nm,', ',end='')

In [27]:
encode_LE(train, test, ['ID','branch_code', 'occupation_code','occupation_category_code','PCODE','sex','marital_status'])

ID , branch_code , occupation_code , occupation_category_code , PCODE , sex , marital_status , 

In [28]:
train.head()

,ID,join_date,sex,marital_status,birth_year,branch_code,occupation_code,occupation_category_code,PCODE,ID X PCODE,Label
0,5384,2019-01-02,0,2,1987,0,19,5,15,4WKQSBB X P5DA,0
1,13834,2019-01-06,0,2,1981,11,19,5,15,CP5S02H X P5DA,0
2,3217,2013-01-06,1,6,1991,3,185,1,15,2YKDILJ X P5DA,0
3,3010,2019-01-08,1,2,1990,0,82,0,15,2S9E81J X P5DA,0
4,12546,2019-01-08,1,2,1990,3,157,5,15,BHDYVFT X P5DA,0


In [29]:
test.head()

,ID,join_date,sex,marital_status,birth_year,branch_code,occupation_code,occupation_category_code,PCODE,ID X PCODE
0,16566,2018-01-12,1,2,1984,4,96,1,15,F86J5PC X P5DA
1,18664,2019-01-10,1,2,1996,0,128,1,15,H6141K3 X P5DA
2,29677,2020-01-01,0,7,1968,11,19,5,15,RBAYUXZ X P5DA
3,22120,2019-01-02,1,2,1989,4,19,5,15,KCBILBQ X P5DA
4,23700,2020-01-02,0,2,1982,11,3,5,15,LSEC1ZJ X P5DA


# Inserting Year, month and day to the dataframe

In [30]:
# insert year, month and day to the dataframe
def insert_YMD(meth):
    meth.fillna(method='pad', inplace=True)
    # Filling the nan values with the previous value
    meth.insert(loc= 2, column= 'year', value=meth['join_date'].apply(lambda x: x.year))
    # Returning the year of the join date
    meth.insert(loc= 2, column= 'month', value=meth['join_date'].apply(lambda x: x.month))
    # Returning the month of the join date
    meth.insert(loc= 2, column= 'day', value=meth['join_date'].apply(lambda x: x.day))
    # Returning the day of the join date
# And everything is inplace

In [31]:
insert_YMD(train)
insert_YMD(test)

In [32]:
test.head()

,ID,join_date,day,month,year,sex,marital_status,birth_year,branch_code,occupation_code,occupation_category_code,PCODE,ID X PCODE
0,16566,2018-01-12,12,1,2018,1,2,1984,4,96,1,15,F86J5PC X P5DA
1,18664,2019-01-10,10,1,2019,1,2,1996,0,128,1,15,H6141K3 X P5DA
2,29677,2020-01-01,1,1,2020,0,7,1968,11,19,5,15,RBAYUXZ X P5DA
3,22120,2019-01-02,2,1,2019,1,2,1989,4,19,5,15,KCBILBQ X P5DA
4,23700,2020-01-02,2,1,2020,0,2,1982,11,3,5,15,LSEC1ZJ X P5DA


# MODEL TRAINING

In [33]:
for date in (train, test):
    date.drop(columns=['join_date'], inplace=True)

In [34]:
train.head(2)

,ID,day,month,year,sex,marital_status,birth_year,branch_code,occupation_code,occupation_category_code,PCODE,ID X PCODE,Label
0,5384,2,1,2019,0,2,1987,0,19,5,15,4WKQSBB X P5DA,0
1,13834,6,1,2019,0,2,1981,11,19,5,15,CP5S02H X P5DA,0


# Splitting data into training and testing set

In [35]:
from sklearn.model_selection import train_test_split

In [36]:
X_train, X_test, y_train, y_test = train_test_split(train.drop(columns=['ID X PCODE', 'Label']), train['Label'], test_size=0.1, random_state=3)
# Splitting the training and the testing set into 90% and 10% respectively with the random state of 42

In [37]:
X_train.head()

,ID,day,month,year,sex,marital_status,birth_year,branch_code,occupation_code,occupation_category_code,PCODE
432274,16107,8,1,2016,1,2,1966,3,131,1,12
559671,24001,12,1,2011,0,5,1961,4,179,1,7
159645,30897,9,1,2016,0,2,1977,4,164,5,8
91109,9722,2,1,2018,1,2,1987,11,3,5,1
45781,33716,5,1,2013,1,6,1958,12,39,5,18


# Testing the performance of each model

In [38]:
from concurrent.futures import ThreadPoolExecutor
from sklearn.metrics import classification_report
def model_testing(md):
    with ThreadPoolExecutor(max_workers=2) as executor:
        executor.submit(md.fit, X_train, y_train)
    print("Accuracy score:", md.score(X_test, y_test), '\n')
    print(classification_report(y_test, md.predict(X_test)))
    return md.predict(test.drop(columns=['ID X PCODE']))

# Using Random forest classifier

In [39]:
rd = RandomForestClassifier(n_estimators=18)

n- 18 - .9582
n- 20 - .-


In [40]:
pred = model_testing(rd)

Accuracy score: 0.9581875837719441 

              precision    recall  f1-score   support

           0       0.97      0.99      0.98     54490
           1       0.88      0.72      0.79      6688

    accuracy                           0.96     61178
   macro avg       0.92      0.85      0.88     61178
weighted avg       0.96      0.96      0.96     61178



In [41]:
test.head(2)

,ID,day,month,year,sex,marital_status,birth_year,branch_code,occupation_code,occupation_category_code,PCODE,ID X PCODE
0,16566,12,1,2018,1,2,1984,4,96,1,15,F86J5PC X P5DA
1,18664,10,1,2019,1,2,1996,0,128,1,15,H6141K3 X P5DA


In [42]:
data= pd.DataFrame(pred)

In [43]:
data.insert(loc=0, column='ID X PCODE', value=test['ID X PCODE'])

In [44]:
data.columns= ['ID X PCODE', 'Label']
data.to_csv('Prediction.csv', index= False)
data.head()

,ID X PCODE,Label
0,F86J5PC X P5DA,0
1,H6141K3 X P5DA,0
2,RBAYUXZ X P5DA,0
3,KCBILBQ X P5DA,0
4,LSEC1ZJ X P5DA,0


In [45]:
data['Label'].value_counts()

0    190813
1     19187
Name: Label, dtype: int64

In [46]:
del data

# Using Logistic regression

In [47]:
from sklearn.linear_model import LogisticRegression

In [48]:
log = LogisticRegression(solver='lbfgs')

In [49]:
pred = model_testing(log)

Accuracy score: 0.8906796560855209 

              precision    recall  f1-score   support

           0       0.89      1.00      0.94     54490
           1       0.00      0.00      0.00      6688

    accuracy                           0.89     61178
   macro avg       0.45      0.50      0.47     61178
weighted avg       0.79      0.89      0.84     61178



# Using Decision Tree

In [50]:
from sklearn.tree import DecisionTreeClassifier

In [51]:
dt = DecisionTreeClassifier()

In [52]:
pred = model_testing(dt)

Accuracy score: 0.9427245088103566 

              precision    recall  f1-score   support

           0       0.97      0.97      0.97     54490
           1       0.74      0.74      0.74      6688

    accuracy                           0.94     61178
   macro avg       0.85      0.85      0.85     61178
weighted avg       0.94      0.94      0.94     61178



# Using Stacking method

In [53]:
from sklearn.ensemble import StackingClassifier

In [54]:
from sklearn.naive_bayes import GaussianNB

In [55]:
from sklearn.neighbors import KNeighborsClassifier

In [59]:
level0 = [('Logistic', LogisticRegression()), ('RandomForest', RandomForestClassifier()), ('DecisionTree', DecisionTreeClassifier()), ('Gaussian', GaussianNB()), ('knn', KNeighborsClassifier())]
level1 = log

In [60]:
model = StackingClassifier(estimators=level0, final_estimator=log, cv=5)

In [61]:
model.fit(X_train, y_train)

StackingClassifier(cv=5,
                   estimators=[('Logistic', LogisticRegression()),
                               ('RandomForest', RandomForestClassifier()),
                               ('DecisionTree', DecisionTreeClassifier()),
                               ('Gaussian', GaussianNB()),
                               ('knn', KNeighborsClassifier())],
                   final_estimator=LogisticRegression())

In [62]:
model.score(X_test, y_test)

0.9609500147111707

In [63]:
gen_pred = model_testing(model)

Accuracy score: 0.9609827062015758 

              precision    recall  f1-score   support

           0       0.97      0.99      0.98     54490
           1       0.89      0.74      0.80      6688

    accuracy                           0.96     61178
   macro avg       0.93      0.86      0.89     61178
weighted avg       0.96      0.96      0.96     61178



In [64]:
gen_data= pd.DataFrame(gen_pred)
gen_data.insert(loc=0, column='ID X PCODE', value=test['ID X PCODE'])

In [65]:
gen_data.columns= ['ID X PCODE', 'Label']
gen_data.to_csv('Prediction.csv', index= False)
gen_data.head()

,ID X PCODE,Label
0,F86J5PC X P5DA,0
1,H6141K3 X P5DA,0
2,RBAYUXZ X P5DA,0
3,KCBILBQ X P5DA,0
4,LSEC1ZJ X P5DA,0
